In [14]:
import os.path
import pandas as pd

filename = 'all.csv'
df = pd.read_csv(filename, converters={'code': str})
print('Done', len(df))

Done 5325


In [15]:
import re

'''
import pandas as pd
data = {
    'code': ['600737'],
    'code_name': ['中粮糖业'],
    '行业': ['食品加工'],
    '最新': [9.59]
}
df = pd.DataFrame(data)
print(df)
filename = 'temp.csv'
'''
YEAR = 2024
df[f'股息率_{YEAR}'] = 0.0
for idx, row in df.iterrows():
    code = row['code']
    #print(idx, code)
    filename_fh = f'./hist_fh/{code}.csv'
    if not os.path.exists(filename_fh):
        print(idx, code, '没有分红记录')
        continue

    try:
        df_fh = pd.read_csv(filename_fh)
        df_fh = df_fh[['报告期', '分红方案说明', 'A股股权登记日', 'A股除权除息日']]
        df_fh = df_fh[df_fh['报告期'].str.contains(f'{YEAR}')]
        #print(df_fh)
    
        #分红派现
        values = []
        for item in df_fh['分红方案说明']:
            match = re.search(r'派(.*?)元', item)
            if match:
                value = match.group(1)
                values.append(float(value))
        values_sum = sum(values)    
        #print(f'{values}, {values_sum}')        
        curr = float(df.at[idx, '最新'])
        df.at[idx, f'股息率_{YEAR}'] = round(values_sum/10/curr*100, 2)
    except Exception as e:
        print(idx, code, e)
        df.drop(idx, inplace=True)

print('Done', len(df))
df.to_csv(filename, index = False)
df.head(3)

Done 5325


,code,name,行业,上市时间,最新,涨幅,股息率_2024,股息率_2023,股息率_2022,股息率_2021,股息率_2020
0,000001,平安银行,银行,19910403,11.46,-0.78,5.31,6.27,2.49,1.99,1.57
1,000002,万科A,房地产开发,19910129,6.63,-0.60,0.00,0.00,10.26,14.72,18.85
2,000004,国华网安,软件开发,19901201,9.09,-3.30,0.00,0.00,0.00,0.00,0.00


In [6]:
df_temp = df[df['股息率_2024'] >= 8]
len(df_temp)

17

In [7]:
df_temp

,code,name,行业,上市时间,最新,涨幅,股息率_2024
359,000915,华特达因,化学制药,19990609,29.97,-0.43,8.34
377,000937,冀中能源,煤炭行业,19990909,6.74,-0.44,11.87
769,002271,东方雨虹,装修建材,20080910,11.08,-0.36,13.76
799,002303,美盈森,包装材料,20091103,3.57,-1.11,15.99
1058,002582,好想你,食品饮料,20110520,11.64,1.13,12.03
1269,002818,富森美,商业百货,20161109,12.84,-1.0,8.41
1392,002951,*ST金时,包装材料,20190315,8.4,-4.65,11.90
3056,600256,广汇能源,石油行业,20000526,5.88,-1.18,10.58
3561,600894,广日股份,专用设备,19960328,10.53,-1.13,12.82
3813,601886,江河集团,装修装饰,20110818,5.52,-0.18,9.96
